In [59]:
import numpy as np
import pandas as pd
from transaction_dates import *
from imiona import *
from collections import Counter
from samochody import *
from czesci import *

# Pracownicy

In [60]:
birth_dates_lims = [1960, 1995]
begining_date = "2014-01-01"
end_date = "2018-01-01"

def rind(a, b):
    return np.random.randint(a, b)

jobs_list = np.array(["menadżer", "sprzedawca", "mechanik", "pomocnik mechanika", 
                       "diagnosta samochodowy", "sprzątający", "informatyk", "księgowy"])
jobs_amount = np.array([1, rind(2, 5), rind(3, 6), rind(1, 3),
                        rind(1, 3), rind(1, 3), rind(1, 3), rind(0, 2)])
fixed_jobs = np.array([0, 1, 2, np.sum(jobs_amount[:2]), np.sum(jobs_amount[0:6])]) # zaczynamy z szefem, dwoma sprzedawcami i mechanikiem

employment_dates = random_transaction_dates(begining_date, end_date, np.sum(jobs_amount))
employment_dates[fixed_jobs] = begining_date  

wages_lims = [(60, 75), (45, 60), (45, 60), (35, 45),
              (47, 60), (35, 40), (40, 60), (40, 53)]

employees_df = pd.DataFrame(columns=["id", "imię", "nazwisko", "pesel",
                                     "miasto", "adres", "telefon", "email", "data_zatrudnienia",
                                     "data_zwolnienia", "stanowisko", "płaca"])
dismissal_dates = [[] for i in range(len(jobs_list))]
for (i, job) in enumerate(jobs_list):

    begin_times = ["" for j in range(jobs_amount[i])]
    for j in range(jobs_amount[i]):
        employment_date = employment_dates[np.sum(jobs_amount[:i]) + j]
        if np.random.rand() <= 0.5:
            first_name, last_name = random_name_man()
            pesel, birth_date = random_pesel_date_man(birth_dates_lims)
            sex = "M"
            city = random_city(prob=0.7)
            address = random_street()
            phone = random_phone()
            #email = random_email(first_name, last_name, birth_date[-4:])
            email = employee_email(first_name, last_name)
            employment_date = employment_dates[np.sum(jobs_amount[:i]) + j]
            wage = np.random.choice(np.arange(wages_lims[i][0], wages_lims[i][1], 0.1))
        else:
            first_name, last_name = random_name_woman()
            pesel, birth_date = random_pesel_date_woman(birth_dates_lims)
            sex = "K"
            city = random_city(prob=0.7)
            address = random_street()
            phone = random_phone()
            #email = random_email(first_name, last_name, birth_date[-4:])
            email = employee_email(first_name, last_name)
            employment_date = employment_dates[np.sum(jobs_amount[:i]) + j]
            wage = np.random.choice(np.arange(wages_lims[i][0], wages_lims[i][1], 0.1))
        begin_times[j] = employment_date

        employees_df.loc[np.sum(jobs_amount[:i]) + j] = [0, first_name, last_name, pesel,
                                                         city, address, phone, email, employment_date,
                                                         None, job, wage]
    dismissal_dates_j = [None for j in range(jobs_amount[i])]
    c = 0
    for j in range(len(begin_times)):
        if len(begin_times) >= 2:
            min_date = min(begin_times[:j] + begin_times[j+1:])
        else:
            min_date = begin_times[j]
        if np.random.rand() <= (jobs_amount[i]-1)/10 * 1/(1+c):
            dismissal_date = random_transaction_dates(str(max(min_date, begin_times[j])), end_date, 1)[0]
            okay = False
            for k in range(len(dismissal_dates_j)):
                if (dismissal_dates_j[k] is not None) and (dismissal_date >=  begin_times[k]) and (dismissal_date <= dismissal_dates_j[k]):
                    okay = True
                    break
                elif dismissal_date >=  begin_times[k]:
                    okay = True
                    break
            if okay:
                dismissal_dates_j[j] = dismissal_date
                c += 1
    dismissal_dates[i] = dismissal_dates_j

diss = []
for i in range(len(dismissal_dates)):
    diss += dismissal_dates[i]
employees_df["data_zwolnienia"] = diss

employees_df = employees_df.sort_values(by="data_zatrudnienia")   
employees_df["id"] =  np.arange(1, len(employees_df)+1)

In [61]:
employees_df

,id,imię,nazwisko,pesel,miasto,adres,telefon,email,data_zatrudnienia,data_zwolnienia,stanowisko,płaca
0,1,Jacek,Olszewski,62051886312,Wrocław,ul. Leszczynowa 81,48090403895,jacek.olszewski@kuba.pl,2014-01-01,None,menadżer,68.7
1,2,Wojciech,Juskowiak,78062634435,Wrocław,ul. Leśna 37/2,48923435716,wojciech.juskowiak@kuba.pl,2014-01-01,None,sprzedawca,55.6
2,3,Roman,Nowak,80081093474,Wrocław,ul. Widokowa 52,48480896676,roman.nowak@kuba.pl,2014-01-01,None,sprzedawca,54.6
3,4,Stanisław,Stanek,95071012250,Wrocław,ul. Cegłowa 86,48298294924,stanislaw.stanek@kuba.pl,2014-01-01,None,mechanik,51.3
11,5,Jan,Kowalczyk,89031702213,Wrocław,ul. Rycerska 10,48787211023,jan.kowalczyk@kuba.pl,2014-01-01,None,informatyk,46.9
4,6,Bożena,Kopietz,85012781505,Wrocław,ul. Urocza 93/46,48823077697,bozena.kopietz@kuba.pl,2015-02-10,None,mechanik,56.6
5,7,Krzysztof,Zanto,91012712636,Wrocław,ul. Kurkowa 7/15,48661177058,krzysztof.zanto@kuba.pl,2015-07-23,2016-08-04,mechanik,55.2
6,8,Tomasz,Shmiel,95012727618,Busko-Zdrój,ul. Orzeszkowej 47/12,48669880817,tomasz.shmiel@kuba.pl,2015-10-24,None,mechanik,49.0
7,9,Jerzy,Burke,78041513232,Gniew,ul. Rolna 55,48098648827,jerzy.burke@kuba.pl,2016-04-08,2017-02-19,pomocnik mechanika,37.5
8,10,Marcelina,Woźniak,79030931040,Wrocław,ul. Tytusa Chałubińskiego 53/10,48522979803,marcelina.wozniak@kuba.pl,2016-07-29,None,pomocnik mechanika,39.0


In [62]:
employees_df.to_csv("outputs/pracownicy.csv", index=False)

# Transakcje

In [63]:
no_transactions = np.random.poisson(2000)
begining_buy = 20

dates = random_transaction_dates(begining_date, end_date, no_transactions)

In [64]:
client_ids = np.zeros(len(dates), dtype=int) # array with client ids for table with transactions

j = 1
for i in range(len(dates)):
    prob = i/(2*len(dates))
    if np.random.rand() <= prob:
        client_ids[i] = np.random.choice(np.unique(client_ids[:i]))
    else:
        client_ids[i] = j
        j += 1

In [65]:
car_ids = np.zeros(len(dates), dtype=int) # array with car ids for table with transactions
car_ids[:begining_buy] = np.arange(1, 21)

client_ids = np.zeros(len(dates), dtype=int) # array with client ids for table with transactions
client_ids[:begining_buy] = np.arange(1, 21)
year_ceil = [int(date[:4]) for date in dates[:20]]

transactions_types = np.zeros(len(dates))

clients_cars = {}
our_cars = np.arange(1, 21).tolist()

jp = 21
jc = 21
for i in range(20, len(dates)):
    weights = np.array([1.5*len(clients_cars)/max(1, len(our_cars)) , len(our_cars)*50, len(clients_cars)/(5*max(1, len(our_cars)))])
    weights = weights/np.sum(weights)
    transaction_type = np.random.choice(np.arange(3), p=weights) # 0-buy, 1-sell, 2-service
    transactions_types[i] = transaction_type
    if transaction_type == 0:
        prob = i/(2*len(dates)) 
        if np.random.rand() <= prob: #client that already is in database
            client_ids[i] = np.random.choice(np.unique(client_ids[:i]))
            if client_ids[i] in clients_cars and np.random.rand() <= 0.6 and len(clients_cars[client_ids[i]]) >= 1:
                car_ids[i] = np.random.choice(clients_cars[client_ids[i]])
                clients_cars[client_ids[i]].remove(car_ids[i])
            else:
                car_ids[i] = jc
                jc += 1
            our_cars.append(car_ids[i])
        else:
            client_ids[i] = jp
            car_ids[i] = jc
            our_cars.append(car_ids[i])
            jp += 1
            jc += 1
            year_ceil.append(int(dates[i][:4])-19)

    if transaction_type == 1:
        prob = i/(2*len(dates))
        if np.random.rand() <= prob: #client that already is in database
            client_ids[i] = np.random.choice(np.unique(client_ids[:i]))
            car_ids[i] = np.random.choice(our_cars)
            if client_ids[i] in clients_cars:
                clients_cars[client_ids[i]].append(car_ids[i])
            else:
                clients_cars[client_ids[i]] = [car_ids[i]]
            our_cars.remove(car_ids[i])
        else:
            client_ids[i] = jp
            car_ids[i] = np.random.choice(our_cars)
            clients_cars[client_ids[i]] = [car_ids[i]]
            our_cars.remove(car_ids[i])
            jp += 1
            year_ceil.append(int(dates[i][:4])-19)
    
    if transaction_type == 2:
        prob = i/(2*len(dates))
        if np.random.rand() <= prob: #client that already is in database
            client_ids[i] = np.random.choice(np.unique(client_ids[:i]))
            if client_ids[i] in clients_cars:
                if np.random.rand() <= 0.6 and len(clients_cars[client_ids[i]]) >= 1:
                    car_ids[i] = np.random.choice(clients_cars[client_ids[i]])
                else:
                    car_ids[i] = jc
                    clients_cars[client_ids[i]].append(car_ids[i])
                    jc += 1
            else:
                car_ids[i] = jc
                clients_cars[client_ids[i]] = [car_ids[i]]
                jc += 1
        else:
            client_ids[i] = jp
            car_ids[i] = jc
            clients_cars[client_ids[i]] = [car_ids[i]]
            jp += 1
            jc += 1
            year_ceil.append(int(dates[i][:4])-19)

In [66]:
len(year_ceil), len(pd.unique(client_ids))

(1480, 1480)

In [67]:
year_ceil

[2014,
 2014,
 2014,
 2014,
 2014,
 2014,
 2014,
 2014,
 2014,
 2014,
 2014,
 2014,
 2014,
 2014,
 2014,
 2014,
 2014,
 2014,
 2014,
 2014,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,

In [68]:
buy_indexes = np.where(transactions_types == 0)
sell_indexes = np.where(transactions_types == 1)
service_indexes = np.where(transactions_types == 2)

In [69]:
transaction_dates_buy = dates[buy_indexes]
transaction_dates_sell = dates[sell_indexes]
transaction_dates_service = dates[service_indexes]

client_ids_buy = client_ids[buy_indexes]
client_ids_sell = client_ids[sell_indexes]
client_ids_service = client_ids[service_indexes]

car_ids_buy = car_ids[buy_indexes]
car_ids_sell = car_ids[sell_indexes]
car_ids_service = car_ids[service_indexes]

In [70]:
buy_employee_id = np.zeros(len(transaction_dates_buy))

for i in range(len(transaction_dates_buy)):
    filter1 = employees_df["stanowisko"] == "sprzedawca"
    filter2 = employees_df["data_zatrudnienia"] <= transaction_dates_buy[i]
    filter3 = pd.notna(employees_df["data_zwolnienia"])
    filter4 = employees_df["data_zwolnienia"] < transaction_dates_buy[i]
    buy_employee_id[i] = np.random.choice(employees_df['id'][filter1 & filter2 & ~(filter3 & filter4)])

In [71]:
sell_employee_id = np.zeros(len(transaction_dates_sell))

for i in range(len(transaction_dates_sell)):
    filter1 = employees_df["stanowisko"] == "sprzedawca"
    filter2 = employees_df["data_zatrudnienia"] <= transaction_dates_sell[i]
    filter3 = pd.notna(employees_df["data_zwolnienia"])
    filter4 = employees_df["data_zwolnienia"] < transaction_dates_sell[i]
    sell_employee_id[i] = np.random.choice(employees_df['id'][filter1 & filter2 & ~(filter3 & filter4)])

In [72]:
service_employee_id = np.zeros(len(transaction_dates_service))

for i in range(len(transaction_dates_service)):
    filter1 = employees_df["stanowisko"] == "mechanik"
    filter2 = employees_df["data_zatrudnienia"] <= transaction_dates_service[i]
    filter3 = pd.notna(employees_df["data_zwolnienia"])
    filter4 = employees_df["data_zwolnienia"] < transaction_dates_service[i]
    service_employee_id[i] = np.random.choice(employees_df['id'][filter1 & filter2 & ~(filter3 & filter4)])


# klienci

In [73]:
year_floor = 1960
transaction_counter = Counter(client_ids)

clients_df = pd.DataFrame(columns=["id", "imię", "nazwisko", "płeć", "PESEL", "miasto", "adres", "karta_lojalnościowa", "telefon", "email"])
unique_clients = np.unique(client_ids)
for i in range(len(unique_clients)):
    if np.random.rand() <= 0.5: # chłop probability
        id_ = unique_clients[i]
        first_name, last_name = random_name_man()
        if id_ in client_ids_buy or id_ in client_ids_sell:
            pesel, birth_date = random_pesel_date_man([year_floor, year_ceil[i]])
            year = birth_date[-4:]
        else:
            pesel = None
            birth_date = None
            year = str(np.random.randint(year_floor, year_ceil[i]))
        sex = 'M'
        city = random_city()
        address = random_street()
        if transaction_counter[id_] >= 3:
            loyalty_card = True
        else:
            loyalty_card = False
        phone = random_phone()
        email = random_email(first_name, last_name, year)
        clients_df.loc[i] = [unique_clients[i], first_name, last_name, sex, pesel, city, address, loyalty_card, phone, email]
    else:
        id_ = unique_clients[i]
        first_name, last_name = random_name_woman()
        if id_ in client_ids_buy or id_ in client_ids_sell:
            pesel, birth_date = random_pesel_date_man([year_floor, year_ceil[i]])
            year = birth_date[-4:]
        else:
            pesel = None
            birth_date = None
            year = str(np.random.randint(year_floor, year_ceil[i]))
        sex = 'K'
        city = random_city()
        address = random_street()
        if transaction_counter[id_] >= 3:
            loyalty_card = True
        else:
            loyalty_card = False
        phone = random_phone()
        email = random_email(first_name, last_name, year)
        clients_df.loc[i] = [unique_clients[i], first_name, last_name, sex, pesel, city, address, loyalty_card, phone, email]
                             
clients_df.astype({"PESEL": str})

,id,imię,nazwisko,płeć,PESEL,miasto,adres,karta_lojalnościowa,telefon,email
0,1,Michał,Guła,M,94020313251,Wrocław,ul. Kolejowa 76,False,48863833306,m_gu@gmail.com
1,2,Edyta,Warycha,K,09280702114,Wrocław,ul. Senatorska 12/46,False,48311358859,warycha5@gmail.com
2,3,Grażyna,Kotliarova,K,10240193655,Łódź,ul. Łączna 34,False,48317122310,gra-kotliar@vp.pl
3,4,Amelia,Zakrzewska,K,80042553410,Wrocław,ul. Chabrowa 14/20,False,48074736377,a_zakrze@wp.pl
4,5,Stefania,Zięba,K,07280439412,Wrocław,ul. Kościuszki 69,False,48235300684,zieb_stef8769@gmail.com
...,...,...,...,...,...,...,...,...,...,...
1475,1476,Maksymilian,Kapusta,M,89120905817,Wrocław,ul. Krótka 33,False,48948802154,qug0mq5s7p9@onet.pl
1476,1477,Szymon,Majcher,M,86041987919,Wrocław,ul. Gospodarcza 4/30,False,48613026760,majcheszymon@vp.pl
1477,1478,Leszek,Urbaniak,M,88102627790,Nowe Miasto Lubawskie,ul. Feliksa Zawadzkiego 33,False,48237719812,urbaniak883@gmail.com
1478,1479,Sławomir,Domagała,M,79071102759,Skarżysko-Kamienna,ul. Marysieńki 31/26,False,48697704026,t8wv81e5w1ue@gmail.com


In [74]:
clients_with_lc = clients_df[clients_df["karta_lojalnościowa"]]["id"].values

loyalty_card_dates = [() for i in range(len(clients_with_lc))]
for (i, client) in enumerate(clients_with_lc):
    loyalty_card_dates[i] = client, dates[np.where(client_ids == client)[0][2]]


In [75]:
clients_df

,id,imię,nazwisko,płeć,PESEL,miasto,adres,karta_lojalnościowa,telefon,email
0,1,Michał,Guła,M,94020313251,Wrocław,ul. Kolejowa 76,False,48863833306,m_gu@gmail.com
1,2,Edyta,Warycha,K,09280702114,Wrocław,ul. Senatorska 12/46,False,48311358859,warycha5@gmail.com
2,3,Grażyna,Kotliarova,K,10240193655,Łódź,ul. Łączna 34,False,48317122310,gra-kotliar@vp.pl
3,4,Amelia,Zakrzewska,K,80042553410,Wrocław,ul. Chabrowa 14/20,False,48074736377,a_zakrze@wp.pl
4,5,Stefania,Zięba,K,07280439412,Wrocław,ul. Kościuszki 69,False,48235300684,zieb_stef8769@gmail.com
...,...,...,...,...,...,...,...,...,...,...
1475,1476,Maksymilian,Kapusta,M,89120905817,Wrocław,ul. Krótka 33,False,48948802154,qug0mq5s7p9@onet.pl
1476,1477,Szymon,Majcher,M,86041987919,Wrocław,ul. Gospodarcza 4/30,False,48613026760,majcheszymon@vp.pl
1477,1478,Leszek,Urbaniak,M,88102627790,Nowe Miasto Lubawskie,ul. Feliksa Zawadzkiego 33,False,48237719812,urbaniak883@gmail.com
1478,1479,Sławomir,Domagała,M,79071102759,Skarżysko-Kamienna,ul. Marysieńki 31/26,False,48697704026,t8wv81e5w1ue@gmail.com


In [76]:
clients_df.to_csv("outputs/klienci.csv", index=False)

In [77]:
np.sum(clients_df['karta_lojalnościowa'])

np.int64(84)

In [78]:
np.sum(pd.isna(clients_df["PESEL"]))

np.int64(71)

# auta

In [79]:
rng = np.random.default_rng()

data_car, data_bike = get_car_data()
make_list_car, model_list_car, car_weights = get_car_weights()
make_list_bike, model_list_bike, bike_weights = get_bike_weights()

In [80]:
def create_car(i):

    is_car = rng.random() < 0.9
    colors = np.array(('czarny', 'biały', 'szary', 'srebrny', 'niebieski', 'czerwony', 'brązowy', 'złoty', 'żółty', 'inny'))
    colors_weights = np.array((0.22, 0.19, 0.18, 0.15, 0.10, 0.09, 0.03, 0.02, 0.01, 0.01))

    if is_car:
        index = rng.choice(len(make_list_car), p=car_weights)
        make, model = make_list_car[index], model_list_car[index]
        weights = data_car[(data_car['make'] == make) & (data_car['model'] == model)]
        body = np.array(('SUV', 'Hatchback', 'Sedan', 'Kombi', 'Coupe', 'Van', 'Minivan', 'Kabriolet'))
        body_weights = np.array((0.25, 0.25, 0.23, 0.17, 0.06, 0.03, 0.009, 0.001))
    else:
        index = rng.choice(len(make_list_bike), p=bike_weights)
        make, model = make_list_bike[index], model_list_bike[index]
        weights = False
        body = False
        body_weights = False

    wheels = 4 if is_car else 2
    year = get_year()
    gear = get_gear(weights, is_car)
    fuel = get_fuel(weights, is_car)
    post_accident = get_post_accident()
    hp = get_hp(weights, is_car)
    color = get_color(colors, colors_weights)
    body_type = get_body_type(body, body_weights, is_car)
    engine_capacity = get_engine_capacity(hp, is_car)
    doors = get_doors(is_car)
    seats = get_num_of_seats(is_car)
    available = i+1 in our_cars
    
    return [i+1, make, model, wheels, year, body_type, post_accident, color, fuel, engine_capacity, hp, gear, doors, seats, available]


def create_car_table(n):
    
    df = pd.DataFrame(columns=['id', 'marka', 'model', 'liczba_kół', 'rok_produkcji', 'typ_nadwozia', 'czy_powypadkowy', 'kolor', 'typ_paliwa', 'pojemność_silnika', 'moc', 'skrzynia_biegów', 'liczba_drzwi', 'liczba_miejsc', 'czy_dostępny'])
    for i in range(n):
        row = create_car(i)
        df.loc[i] = row
    return df


In [81]:
df_samochody = create_car_table(max(car_ids))

In [82]:
df_samochody

,id,marka,model,liczba_kół,rok_produkcji,typ_nadwozia,czy_powypadkowy,kolor,typ_paliwa,pojemność_silnika,moc,skrzynia_biegów,liczba_drzwi,liczba_miejsc,czy_dostępny
0,1,Skoda,Octavia,4,2004,SUV,True,niebieski,Benzyna,2.4,150,Automatyczna,5,2,False
1,2,BMW,X2,4,2015,SUV,False,szary,Diesel,2.1,140,Automatyczna,3,7,False
2,3,Opel,Zafira Tourer,4,2014,Sedan,False,srebrny,Diesel,1.9,140,Manualna,5,5,False
3,4,Chevrolet,Aveo,4,2006,Sedan,False,czarny,Benzyna,1.6,101,Manualna,5,5,False
4,5,Nissan,Qashqai,4,2013,Sedan,False,szary,Diesel,2.1,141,Manualna,5,5,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
981,982,Hyundai,i20,4,2010,SUV,False,czarny,Benzyna,1.7,120,Manualna,5,5,False
982,983,Ford,Fiesta,4,2012,Hatchback,True,srebrny,Benzyna,0.8,60,Manualna,3,5,True
983,984,Volkswagen,Tiguan,4,2010,Minivan,True,czarny,Diesel,2.3,150,Manualna,5,5,False
984,985,Skoda,Octavia,4,2010,Sedan,False,szary,Benzyna,2.2,150,Automatyczna,5,2,False


In [83]:
df_samochody.to_csv("outputs/samochody.csv", index=False)

In [84]:
pd.unique(df_samochody["skrzynia_biegów"])

array(['Automatyczna', 'Manualna', 'Półautomatyczna'], dtype=object)

# Usługi

In [85]:
service_employee = rng.choice(employees_df[employees_df['stanowisko'] == 'mechanik']['id'], len(car_ids_service))
service_ids = np.arange(len(transaction_dates_service)) + 1

df_services = pd.DataFrame()
df_services['id'] = service_ids
df_services['id_samochodu'] = car_ids_service
df_services['id_klienta'] = client_ids_service
df_services['id_pracownika'] = service_employee
df_services['data_zaksięgowania'] = transaction_dates_service

# Zakup samochodów

In [86]:
def car_purchase_price(hp, yr_bought, yr_prod, post_acc):
    return np.round(hp * 1000 / np.sqrt(int(yr_bought) - int(yr_prod) + 1) * (0.75 if post_acc else 1) * rng.uniform(0.8, 1.2), -2)

In [87]:
buy_employee = rng.choice(employees_df[employees_df['stanowisko'] == 'sprzedawca']['id'], len(car_ids_buy))
buy_ids = np.arange(len(transaction_dates_buy)) + 1

df_buy = pd.DataFrame()
df_buy['id'] = buy_ids
df_buy['id_samochodu'] = car_ids_buy
df_buy['id_klienta'] = client_ids_buy
df_buy['id_pracownika'] = buy_employee
df_buy['data_zaksięgowania'] = transaction_dates_buy

In [88]:

service_date_fix = pd.merge(df_samochody[['id', 'rok_produkcji']], df_services[['id_samochodu', 'data_zaksięgowania']], how='right', left_on='id', right_on='id_samochodu')
service_date_fix['data_zaksięgowania'] = service_date_fix['data_zaksięgowania'].str[:4]

buy_date_fix = pd.merge(df_samochody[['id', 'rok_produkcji']], df_buy[['id_samochodu', 'data_zaksięgowania']], how='right', left_on='id', right_on='id_samochodu')
buy_date_fix['data_zaksięgowania'] = buy_date_fix['data_zaksięgowania'].str[:4]

for i in range(len(buy_date_fix)):
    if str(buy_date_fix['rok_produkcji'].loc()[i]) >= str(buy_date_fix['data_zaksięgowania'].loc()[i]):
        car_index = buy_date_fix.loc[i, 'id_samochodu']
        df_samochody.loc[car_index-1, 'rok_produkcji'] = str(int(df_samochody.loc[car_index-1, 'rok_produkcji']) - 10)

for i in range(len(service_date_fix)):
    if str(service_date_fix['rok_produkcji'].loc()[i]) >= str(service_date_fix['data_zaksięgowania'].loc()[i]):
        car_index = service_date_fix.loc[i, 'id_samochodu']
        df_samochody.loc[car_index-1, 'rok_produkcji'] = str(int(df_samochody.loc[car_index-1, 'rok_produkcji']) - 10)

C:\Users\oskar.LAPTOP-KARLNF81\AppData\Local\Temp\ipykernel_7648\3526569416.py:10: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '2005' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df_samochody.loc[car_index-1, 'rok_produkcji'] = str(int(df_samochody.loc[car_index-1, 'rok_produkcji']) - 10)


In [89]:
purchase_prices = []
for j, i in enumerate(car_ids_buy):
    row = df_samochody.loc()[i-1]
    year_bought = transaction_dates_buy[j][:4]
    purchase_prices.append(car_purchase_price(row['moc'], year_bought, row['rok_produkcji'], row['czy_powypadkowy']))

df_buy['cena'] = purchase_prices

In [90]:
df_buy

,id,id_samochodu,id_klienta,id_pracownika,data_zaksięgowania,cena
0,1,1,1,3,2014-01-02,29700.0
1,2,2,2,2,2014-01-03,41500.0
2,3,3,3,3,2014-01-03,39100.0
3,4,4,4,2,2014-01-04,39300.0
4,5,5,5,2,2014-01-04,94100.0
...,...,...,...,...,...,...
920,921,984,1137,3,2017-12-27,33400.0
921,922,985,1477,3,2017-12-28,59800.0
922,923,986,922,2,2017-12-29,12500.0
923,924,46,95,3,2017-12-29,34900.0


In [91]:
df_buy.to_csv("outputs/zakup_samochodu.csv", index=False)

# Sprzedaż samochodów

In [92]:
sell_employee = rng.choice(employees_df[employees_df['stanowisko'] == 'sprzedawca']['id'], len(car_ids_sell))
sell_ids = np.arange(len(transaction_dates_sell)) + 1

df_sell = pd.DataFrame()
df_sell['id'] = sell_ids
df_sell['id_samochodu'] = car_ids_sell
df_sell['id_klienta'] = client_ids_sell
df_sell['id_pracownika'] = sell_employee
df_sell['data_zaksięgowania'] = transaction_dates_sell

In [93]:
loyalty_card_customer_ids = [el[0] for el in loyalty_card_dates]
loyalty_card_dates_sep = [el[1] for el in loyalty_card_dates]

In [95]:
sell_prices = []
for i in car_ids_sell:
    b_price = df_buy[df_buy['id_samochodu'] == i]['cena'].values[0]
    sell_prices.append(np.round(b_price * rng.uniform(1.2, 1.5), -2))

for i in range(len(sell_ids)):
    customer_id = df_sell.loc()[i]['id_klienta']
    if clients_df[clients_df['id'] == customer_id]['karta_lojalnościowa'].bool() and customer_id in loyalty_card_customer_ids:
        date_index = loyalty_card_customer_ids.index(customer_id)
        if loyalty_card_dates_sep[date_index] < df_sell.loc()[i]['data_zaksięgowania']:
            sell_prices[i] *= 0.97
            #print(i)

sell_installments = []
for i in range(len(sell_ids)):
    if sell_prices[i] > 80000:
        inst_prob = [0.1, 0.2, 0.3, 0.4]
    else:
        inst_prob = [0.4, 0.3, 0.2, 0.1]
    installments = rng.choice([1, 6, 12, 18], p=inst_prob)
    if installments == 6:
        sell_prices[i] *= 1.02
    elif installments == 12:
        sell_prices[i] *= 1.04
    elif installments == 18:
        sell_prices[i] *= 1.06
    sell_installments.append(installments)
#print(sell_installments)

df_sell['cena'] = sell_prices
df_sell['liczba_rat'] = sell_installments
df_sell = df_sell[['id', 'id_samochodu', 'id_klienta', 'id_pracownika', 'cena', 'liczba_rat', 'data_zaksięgowania']]

C:\Users\oskar.LAPTOP-KARLNF81\AppData\Local\Temp\ipykernel_7648\3251071490.py:8: FutureWarning: Series.bool is now deprecated and will be removed in future version of pandas
  if clients_df[clients_df['id'] == customer_id]['karta_lojalnościowa'].bool() and customer_id in loyalty_card_customer_ids:


570
900
[np.int64(18), np.int64(1), np.int64(12), np.int64(6), np.int64(1), np.int64(6), np.int64(6), np.int64(18), np.int64(1), np.int64(12), np.int64(18), np.int64(6), np.int64(18), np.int64(12), np.int64(6), np.int64(12), np.int64(12), np.int64(6), np.int64(1), np.int64(18), np.int64(1), np.int64(6), np.int64(6), np.int64(12), np.int64(12), np.int64(18), np.int64(12), np.int64(12), np.int64(1), np.int64(1), np.int64(12), np.int64(1), np.int64(18), np.int64(6), np.int64(6), np.int64(12), np.int64(1), np.int64(12), np.int64(18), np.int64(1), np.int64(1), np.int64(1), np.int64(12), np.int64(6), np.int64(12), np.int64(12), np.int64(6), np.int64(1), np.int64(12), np.int64(1), np.int64(1), np.int64(1), np.int64(1), np.int64(12), np.int64(18), np.int64(1), np.int64(1), np.int64(1), np.int64(6), np.int64(1), np.int64(12), np.int64(18), np.int64(12), np.int64(1), np.int64(12), np.int64(12), np.int64(12), np.int64(6), np.int64(12), np.int64(1), np.int64(12), np.int64(1), np.int64(12), np.int6

In [ ]:
df_sell

,id,id_samochodu,id_klienta,id_pracownika,cena,liczba_rat,data_zaksięgowania
0,1,18,21,2,45900.0,1,2014-01-22
1,2,12,22,6,39066.0,6,2014-01-23
2,3,13,23,3,86496.0,6,2014-01-24
3,4,3,24,2,60300.0,1,2014-01-24
4,5,11,25,2,25700.0,1,2014-01-28
...,...,...,...,...,...,...,...
955,956,999,1542,3,125154.0,6,2017-12-24
956,957,996,1539,2,66038.0,18,2017-12-24
957,958,52,1545,6,45700.0,1,2017-12-27
958,959,980,743,6,63200.0,1,2017-12-27


In [ ]:
np.sum(df_sell["cena"] == float('inf')) # wychodzą inf wiec trzeba poprawić

np.int64(0)

In [ ]:
df_sell.to_csv("outputs/sprzedaż_samochodu.csv", index=False)

# Użyte części

In [ ]:
data_parts = get_data_parts()
data_parts_filtered = data_parts[data_parts['motor'] == True]

def get_parts_indexes(car_id):
    if df_samochody.loc[car_id-1].values[3] == 2:
        parts_set = data_parts_filtered[['id','część']]
    else:
        parts_set = data_parts[['id','część']]
    
    parts_indexes = parts_set['id']
    num_parts = rng.choice([1, 2, 3], p=[0.6, 0.3, 0.1])
    indexes = rng.choice(parts_indexes, size=num_parts)
    return indexes

all_car_parts_indexes = [(i, get_parts_indexes(i)) for i in car_ids_service]
print(all_car_parts_indexes)

[(np.int64(21), array([58, 15])), (np.int64(30), array([62])), (np.int64(32), array([49])), (np.int64(35), array([13,  1])), (np.int64(45), array([16])), (np.int64(50), array([13, 16])), (np.int64(55), array([58])), (np.int64(79), array([24])), (np.int64(96), array([31])), (np.int64(99), array([46])), (np.int64(102), array([59])), (np.int64(103), array([51,  4, 51])), (np.int64(118), array([12, 26])), (np.int64(134), array([61])), (np.int64(135), array([39])), (np.int64(159), array([57])), (np.int64(172), array([51])), (np.int64(175), array([26])), (np.int64(176), array([28, 47])), (np.int64(179), array([63, 15])), (np.int64(191), array([39, 51])), (np.int64(194), array([36, 33])), (np.int64(200), array([61, 11])), (np.int64(201), array([29, 27])), (np.int64(209), array([54])), (np.int64(217), array([54])), (np.int64(255), array([17])), (np.int64(256), array([37, 57])), (np.int64(269), array([45])), (np.int64(282), array([12, 61])), (np.int64(283), array([25])), (np.int64(284), array([

In [ ]:
#df_samochody.loc()[1] - samochód o indeksie 0

def create_part(id, service_id, car):
    part = data_parts.loc()[id]
    if type(part['atrybuty']) != float:
        attributes = part['atrybuty'].split()
        for i in range(len(attributes)):
            if attributes[i] == 'marka':
                attributes[i] = car['marka']
            if attributes[i] == 'model':
                attributes[i] = car['model']
            if attributes[i] == 'kolor':
                attributes[i] = car['kolor']
            if attributes[i] == 'rozmiar':
                attributes[i] = 'R' + str(rng.choice([15, 16, 17]))
            if attributes[i] == 'material':
                attributes[i] = rng.choice(['skóra', 'materiał'], p=[0.2, 0.8])
            if attributes[i] == 'pojemnosc':
                attributes[i] = str(car['pojemność_silnika'])
            if attributes[i] == 'skrzynia':
                attributes[i] = str(car['skrzynia_biegów'])
        price = int(np.floor(part['cena']*car['pojemność_silnika']/1.8 * rng.uniform(0.8, 1.2)))
    else:
        attributes = ''
        price = part['cena']
    quantity = part['maxcount']
    return part['część'], ' '.join(attributes), price, service_id, quantity

parts = []
for service_id, element in enumerate(all_car_parts_indexes):
    car_id, parts_id = element
    car = df_samochody.loc()[car_id-1]
    for part_id in parts_id:
        parts.append(create_part(part_id, service_id+1, car))

parts = np.array(parts)
parts_df = pd.DataFrame(parts, columns=['nazwa', 'atrybuty', 'cena', 'id_usługi', 'ilość']).drop_duplicates(['nazwa', 'atrybuty'])
parts_df['id'] = np.arange(1, len(parts_df)+1)
parts_df = parts_df[['id', 'nazwa', 'atrybuty', 'cena', 'id_usługi', 'ilość']]

parts = pd.DataFrame(parts, columns=['nazwa', 'atrybut', 'cena', 'id_usługi', 'ilość'])

In [ ]:
parts

,nazwa,atrybut,cena,id_usługi,ilość
0,Sprzęgło,Volkswagen Caddy,318,1,1
1,Radio,Volkswagen Caddy,67,1,1
2,Lusterko zewnętrzne prawe,Volkswagen Golf czerwony,25,2,1
3,Filtr paliwa,Audi A4,962,3,1
4,Lakier,biały,274,4,1
...,...,...,...,...,...
163,Pompa paliwowa,Hyundai i30,591,113,1
164,Zderzak,Mazda 3 czerwony,241,114,1
165,Sprzęgło,smart forTwo,187,115,1
166,Rura wydechowa,,100,116,1


# Wyposażenie

In [ ]:
eq_len = len(parts_df)
equipment_id = parts_df['id']
equipment_name = parts_df['nazwa']
equipment_attr = parts_df['atrybuty']
equipment_price = parts_df['cena']

df_equipment = pd.DataFrame()
df_equipment['id'] = equipment_id
df_equipment['nazwa'] = equipment_name
df_equipment['atrybut'] = equipment_attr
df_equipment['cena_części'] = equipment_price.astype(float)

In [ ]:
df_equipment

,id,nazwa,atrybut,cena_części
0,1,Sprzęgło,Volkswagen Caddy,318.0
1,2,Radio,Volkswagen Caddy,67.0
2,3,Lusterko zewnętrzne prawe,Volkswagen Golf czerwony,25.0
3,4,Filtr paliwa,Audi A4,962.0
4,5,Lakier,biały,274.0
...,...,...,...,...
162,134,Sprzęgło,Skoda Fabia,331.0
163,135,Pompa paliwowa,Hyundai i30,591.0
164,136,Zderzak,Mazda 3 czerwony,241.0
165,137,Sprzęgło,smart forTwo,187.0


# Usługi - uzupełnienie

In [ ]:
service_prices = np.zeros(len(service_ids))
for i in range(len(parts)):
    row = parts.loc()[i]
    service_index = int(row['id_usługi'])
    service_prices[service_index-1] += float(row['cena']) * int(row['ilość'])

df_services['cena'] = np.round(service_prices*rng.uniform(1.1, 1.4), -1).astype(int)

In [ ]:
df_services

,id,id_samochodu,id_klienta,id_pracownika,data_zaksięgowania,cena
0,1,21,41,8,2014-02-08,450
1,2,30,57,7,2014-02-22,30
2,3,32,59,4,2014-02-23,1120
3,4,35,64,8,2014-02-26,800
4,5,45,82,8,2014-03-13,190
...,...,...,...,...,...,...
112,113,983,1525,4,2017-12-06,690
113,114,984,1526,8,2017-12-07,280
114,115,993,228,4,2017-12-16,220
115,116,998,1539,8,2017-12-22,120


In [ ]:
df_services.to_csv("outputs/usługi.csv", index=False)

# Użyte części - uzupełnienie

In [ ]:
used_parts_df = parts.merge(df_equipment, on=['nazwa', 'atrybut'])
used_parts_df.rename(columns={'id': 'id_części'}, inplace=True)
used_parts_df['ilość'] = pd.to_numeric(used_parts_df['ilość'])
ids = np.arange(len(used_parts_df))+1
used_parts_df['id'] = ids
used_parts_df = used_parts_df[['id', 'id_usługi', 'id_części', 'ilość']]

In [ ]:
used_parts_df

,id,id_usługi,id_części,ilość
0,1,1,1,1
1,2,1,2,1
2,3,2,3,1
3,4,3,4,1
4,5,4,5,1
...,...,...,...,...
163,164,113,135,1
164,165,114,136,1
165,166,115,137,1
166,167,116,38,1


In [ ]:
used_parts_df.to_csv("outputs/użyte_części.csv", index=False)

# Zakup części

In [ ]:
def get_service_date(part_id):
    
    index_used_part = used_parts_df[used_parts_df['id_części'] == part_id].index[0]
    service_date = df_services[df_services['id'] == index_used_part]['data_zaksięgowania']
    return service_date
    

buy_parts_quantity = used_parts_df.groupby(['id_części'])['ilość'].sum().reset_index()
parts_purchase_pd = pd.DataFrame(columns=['id', 'id_części', 'koszt', 'ilość'])

for i in range(len(buy_parts_quantity)):
    part_id = buy_parts_quantity['id_części'][i]
    quantity = buy_parts_quantity['ilość'][i]
    price = df_equipment['cena_części'][df_equipment['id'] == part_id].to_numpy()
    if price > 1000:
        quantity += rng.random() < 0.5
    elif price > 500:
        quantity += rng.choice([0, 1, 2], p=[0.2, 0.5, 0.3])
    else:
        quantity += rng.binomial(25, 0.15)
    total_cost = np.floor(quantity * price * rng.uniform(0.75, 0.95))[0]
    service_date = get_service_date(part_id)
    
    row = [int(i+1), int(part_id), total_cost, int(quantity)]
    parts_purchase_pd.loc[i] = row


In [ ]:
parts_purchase_pd

,id,id_części,koszt,ilość
0,1.0,1.0,1350.0,5.0
1,2.0,2.0,310.0,5.0
2,3.0,3.0,105.0,5.0
3,4.0,4.0,1814.0,2.0
4,5.0,5.0,626.0,3.0
...,...,...,...,...
133,134.0,134.0,309.0,1.0
134,135.0,135.0,487.0,1.0
135,136.0,136.0,1793.0,8.0
136,137.0,137.0,1242.0,8.0


In [ ]:
parts_purchase_pd.to_csv("outputs/zakup_części.csv", index=False)

In [ ]:
df_equipment = df_equipment[['id', 'nazwa', 'atrybut']]

In [ ]:
df_equipment.to_csv("outputs/wyposażenie.csv", index=False)